<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import matplotlib
import pandas as pd
import numpy as np
import cvxpy as cp
from cvxopt import matrix, solvers
import pickle
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from colorama import Fore
from ipynb.fs.full.config import RISK_FREE_RATE, DATAPATH, EXPECTED_RETURN, STOCKS_NUMBER, MONTO_CARLO_TIMES

In [4]:
class InvestmentStrategy:

    @staticmethod
    def process_data_x_matrix(datapath):
        df_raw = pd.read_excel(datapath)
        df_raw = df_raw.T
        df_raw = df_raw.drop(index=['code'], columns=[0])
        df_raw = df_raw.fillna(method='ffill')
        df = df_raw.fillna(method='backfill')
        return df

    @staticmethod
    def process_data_contain_sp500(datapath):
        df_raw = pd.read_excel(datapath)
        df_raw = df_raw.T
        df_raw = df_raw.drop(index=['code'])
        df_raw = df_raw.fillna(method='ffill')
        df = df_raw.fillna(method='backfill')
        return df

    @staticmethod
    def day_yield_compute(x_matrix):
        day_yield = (x_matrix.shift(-1) - x_matrix) / x_matrix
        return day_yield.iloc[:-1, :]

    @staticmethod
    def ex_vector_compute(x_matrix):
        day_yield = (x_matrix.shift(-1) - x_matrix) / x_matrix
        day_avg_yield = day_yield.mean().to_numpy()
        return day_yield.iloc[:-1, :], day_avg_yield

    @staticmethod
    def ex_matrix_compute(x_matrix, ex_numpy_vector):
        ex_np = np.repeat(np.expand_dims(ex_numpy_vector, axis=0), x_matrix.shape[0], axis=0)
        ex_matrix = pd.DataFrame(ex_np, index=x_matrix.index, columns=x_matrix.columns)
        return ex_matrix

    @staticmethod
    def cov_matrix_compute(x_ex_matrix):
        return np.matmul(x_ex_matrix.T.to_numpy(), x_ex_matrix.to_numpy()) / (x_ex_matrix.shape[0] - 1)

    def compute_weight(self, x_matrix, total_days=252, method="Markowitz", starttime=0, endtime=0):
        total_days_every_year = total_days / 5

        day_yield_matrix, ex_numpy_vector = self.ex_vector_compute(x_matrix)
        ex_matrix = self.ex_matrix_compute(day_yield_matrix, ex_numpy_vector)
        x_ex_matrix = day_yield_matrix - ex_matrix
        cov_matrix_numpy = self.cov_matrix_compute(x_ex_matrix)

        # stocks_number = 50
        n = STOCKS_NUMBER

        one_matrix = np.ones((1, n))
        
        if method == "Markowitz":
            print("\033[0;36;m Initial Calculation of Portfolio Weights Using the Strategy：\033[0m \033[0;34;m Markowitz Portfolio \033[0m")
            annual_yield_vector = ex_numpy_vector * total_days_every_year
            w = cp.Variable(n)
            prob = cp.Problem(cp.Minimize((1 / 2) * cp.quad_form(w, cov_matrix_numpy)),
                              [annual_yield_vector.T @ w == EXPECTED_RETURN,
                               one_matrix @ w == 1])
            prob.solve()

            print("\033[0;36;m Complete Markowitz Portfolio Optimal Weight Quadratic Programming Solution with the Minimum Variance Value：\033[0m \033[0;34;m {} \033[0m".format(prob.value))

            return w.value

        r_p_list = []
        sigma_p_list = []
        sharpe_ratio_list = []
        weight_list = []
        if method == "MontoCarlo":
            print("\033[0;36;m Start Calculation of Portfolio Weights Using the Strategy：\033[0m \033[0;34;m Monto Carlo Solve for the Maximum Sharpe Ratio \033[0m")
            np.random.seed(1)
            risk_free_rate_day = RISK_FREE_RATE / total_days_every_year
            bar = tqdm(list(range(int(MONTO_CARLO_TIMES))),
                       bar_format='{l_bar}%s{bar}%s{r_bar}' % (Fore.BLUE, Fore.RESET))
            for _ in bar:
                weights = np.random.normal(1 / n, 1.0, n - 1)
                weights_last = 1 - np.sum(weights)
                weights = np.append(weights, weights_last)
                weights_row_vector = np.expand_dims(weights, axis=0)
                yield_avg_vector = np.expand_dims(ex_numpy_vector, axis=0)
                sigma_p = np.sqrt(np.matmul(np.matmul(weights_row_vector, cov_matrix_numpy), weights_row_vector.T))[0][
                    0]
                r_p = np.matmul(weights_row_vector, yield_avg_vector.T)[0][0]

                sharpe_ratio = (r_p - risk_free_rate_day) / sigma_p

                r_p_list.append(r_p)
                sigma_p_list.append(sigma_p)
                sharpe_ratio_list.append(sharpe_ratio)
                weight_list.append(weights)

            r_p_list_numpy = np.array(r_p_list)
            sigma_p_list_numpy = np.array(sigma_p_list)
            sharpe_ratio_list_numpy = np.array(sharpe_ratio_list)
            weight_list_numpy = np.array(weight_list)

            # max sharpe ratio
            max_sharpe_ratio = np.max(sharpe_ratio_list_numpy)
            max_sharpe_ratio_index = np.argmax(sharpe_ratio_list_numpy)

            # cov and mean
            sigma_rp = [sigma_p_list_numpy[max_sharpe_ratio_index], r_p_list_numpy[max_sharpe_ratio_index]]

            # alpha 
            alpha = (EXPECTED_RETURN / total_days_every_year - sigma_rp[1]) / (risk_free_rate_day - sigma_rp[1])
            weight_list_numpy_opt_alpha = np.append(weight_list_numpy[max_sharpe_ratio_index], alpha)

            print("\033[0;36;m finished Monto Carlo Strategy Weight Computation \033[0m")
            # plot fig
            filename = os.path.join(os.getcwd(), 'images1')
            if not os.path.exists(filename):
                os.makedirs(filename)
            plt.figure(figsize=(8, 6))
            plt.style.use('seaborn-dark')
            plt.rcParams['savefig.dpi'] = 300  
            plt.rcParams['figure.dpi'] = 300  
            plt.rcParams['axes.unicode_minus'] = False  

            plt.scatter(sigma_p_list_numpy, r_p_list_numpy, c=r_p_list_numpy / sigma_p_list_numpy,
                        marker='o', cmap='coolwarm')
            plt.plot([0, sigma_rp[0]], [risk_free_rate_day, sigma_rp[1]], 'r')
            
            plt.annotate('max Sharpe ratio:{}'.format(max_sharpe_ratio), xy=sigma_rp)
            plt.xlabel('Std. by Day')
            plt.ylabel('Yield by Day')
            plt.colorbar(label='Sharpe ratio')
            plt.title("Monta Carlo Sampling No.{} times and Obtain CAL.".format(MONTO_CARLO_TIMES))
            plt.savefig("./images1/Montacarlo_CAL_{}_{}_{}".format(MONTO_CARLO_TIMES, starttime, endtime), dpi=300)
            print("\033[0;36;m Complete Capital Market Line Plotting. \033[0m")
            return weight_list_numpy_opt_alpha

    @staticmethod
    def get_six_month_map(x_matrix):
        dfx = pd.DataFrame(x_matrix.index, columns=['time'])
        dfx["year"] = pd.to_datetime(pd.DataFrame(x_matrix.index, columns=['time'])['time'], format='%Y-%m-%d').dt.year
        dfx["month"] = pd.to_datetime(pd.DataFrame(x_matrix.index, columns=['time'])['time'],
                                      format='%Y-%m-%d').dt.month

        dfx['yearmonth'] = dfx.apply(lambda r: r['time'][:-2], axis=1)
        dfx = dfx.drop_duplicates(['yearmonth'])

        index_six_month = dfx[(dfx['month'] == 1) | (dfx['month'] == 7)].index.tolist()
        index_slice = int(len(index_six_month) / 2)

        compare_list1 = index_six_month[index_slice:]
        compare_list2 = compare_list1[1:]
        compare_list2.append(x_matrix.shape[0])
        compare_list = list(zip(compare_list1, compare_list2))

        six_map = {k: v for k, v in zip(index_six_month[index_slice:], index_six_month[:index_slice])}
        return six_map, compare_list

    def save_weights_markowitz(self):
        x_matrix_total = self.process_data_x_matrix(DATAPATH)
        six_map, compare_list = self.get_six_month_map(x_matrix_total)

        weight_list = []
        bar = tqdm(six_map.items(), bar_format='{l_bar}%s{bar}%s{r_bar}' % (Fore.BLUE, Fore.RESET))
        for k, v in bar:
            start_time = x_matrix_total.iloc[v:k, :].index[0]
            end_time = x_matrix_total.iloc[v:k, :].index[-1]
            bar.set_description(f"Computing{start_time}--{end_time}weights")
            df_weight = x_matrix_total.iloc[v:k, :]
            total_days = k - v
            weight = self.compute_weight(df_weight, total_days)
            weight_list.append(weight)
        # save weight
        filename = os.path.join(os.getcwd(), 'weights')
        if not os.path.exists(filename):
            os.makedirs(filename)
        with open('./weights/weights_Markowitz.pickle', 'wb') as f:
            pickle.dump(weight_list, f)
        with open('./weights/weights_Markowitz.txt', 'w') as f2:
            f2.write(str(weight_list))
        print("\033[0;36;m weight save completed \033[0m")

    def save_weights_montocarlo(self):
        x_matrix_total = self.process_data_x_matrix(DATAPATH)
        six_map, compare_list = self.get_six_month_map(x_matrix_total)

        weight_list = []
        bar = tqdm(six_map.items(), bar_format='{l_bar}%s{bar}%s{r_bar}' % (Fore.BLUE, Fore.RESET))
        for k, v in bar:
            df_weight = x_matrix_total.iloc[v:k, :]
            total_days = k - v
            start_time = x_matrix_total.iloc[v:k, :].index[0]
            end_time = x_matrix_total.iloc[v:k, :].index[-1]
            bar.set_description(f"Computing{start_time}--{end_time}weights")

            weight = self.compute_weight(df_weight, total_days, method="MontoCarlo", starttime=start_time,
                                         endtime=end_time)
            weight_list.append(weight)
        # save weight
        filename = os.path.join(os.getcwd(), 'weights')
        if not os.path.exists(filename):
            os.makedirs(filename)
        with open('./weights/weights_MontoCarlo.pickle', 'wb') as f:
            pickle.dump(weight_list, f)
        with open('./weights/weights_MontoCarlo.txt', 'w') as f2:
            f2.write(str(weight_list))
        print("\033[0;36;m saved \033[0m")

    def compare_performance(self, method="Markowitz"):
        print("\033[0;36;m Start Comparison with S&P 500 Performance, Using the Comparison Strategy is \033[0m \033[0;34;m {} \033[0m".format(method))
        total_compare_matrix = pd.DataFrame(columns=['SP500', 'Portfolio', "Period"])
        total_compare_matrix_convert_one = pd.DataFrame(columns=['SP500', 'Portfolio', "Period"])
        x_matrix_total_sp500 = self.process_data_contain_sp500(DATAPATH)
        six_map, compare_list = self.get_six_month_map(x_matrix_total_sp500)
        x_matrix_total_sp500_convert_one = x_matrix_total_sp500.div(x_matrix_total_sp500.iloc[compare_list[0][0]],
                                                                    axis='columns')

        if method == "MontoCarlo_alpha0":
            with open('./weights/weights_{}.pickle'.format("MontoCarlo"), 'rb') as f:
                weight_list = pickle.load(f)
        else:
            with open('./weights/weights_{}.pickle'.format(method), 'rb') as f:
                weight_list = pickle.load(f)

        alpha = 0

        for index, period in enumerate(compare_list):
            if method == "Markowitz":
                weights = weight_list[index]

            elif method == "MontoCarlo":
                weights = weight_list[index][:-1]
                alpha = weight_list[index][-1]

            elif method == "MontoCarlo_alpha0":
                weights = weight_list[index][:-1]

            if period[1] != x_matrix_total_sp500.shape[0]:
                period_day_yield_matrix = x_matrix_total_sp500.iloc[period[0]:period[1] + 1, :]
                period_day_matrix_convert_one = x_matrix_total_sp500_convert_one.iloc[period[0]:period[1] + 1, :]
            else:
                period_day_yield_matrix = x_matrix_total_sp500.iloc[period[0]:period[1], :]
                period_day_matrix_convert_one = x_matrix_total_sp500_convert_one.iloc[period[0]:period[1], :]
            day_yield_compare_matrix = self.day_yield_compute(period_day_yield_matrix)
            period_day_matrix_convert_one = period_day_matrix_convert_one.iloc[:-1, :]
            start_time = day_yield_compare_matrix.index[0]
            end_time = day_yield_compare_matrix.index[-1]

            weighted_day_yield = (1 - alpha) * (
                np.matmul(day_yield_compare_matrix.iloc[:, 1:].to_numpy(), weights)) + alpha * RISK_FREE_RATE / 242
            weighted_day_convert_one = np.matmul(period_day_matrix_convert_one.iloc[:, 1:].to_numpy(), weights)

            day_yield_compare_matrix['Portfolio'] = pd.DataFrame(weighted_day_yield,
                                                                 index=day_yield_compare_matrix.index)

            period_day_matrix_convert_one['Portfolio'] = pd.DataFrame(weighted_day_convert_one,
                                                                      index=day_yield_compare_matrix.index)
            day_yield_compare_matrix.rename(columns={0: 'SP500'}, inplace=True)
            period_day_matrix_convert_one.rename(columns={0: 'SP500'}, inplace=True)
            period_series = pd.to_datetime(
                pd.DataFrame(day_yield_compare_matrix.index, columns=['time'])['time'], format='%Y-%m-%d')
            dict_data = {'time': period_series.values}

            day_yield_compare_matrix["Period"] = pd.DataFrame(dict_data, index=day_yield_compare_matrix.index)
            period_day_matrix_convert_one["Period"] = pd.DataFrame(dict_data, index=day_yield_compare_matrix.index)

            # 作图和记录平均收益比较
            sp500_mean = np.mean(day_yield_compare_matrix['SP500'].to_numpy())
            portfolio_mean = np.mean(day_yield_compare_matrix['Portfolio'].to_numpy())
            if sp500_mean < portfolio_mean:
                win = 'Portfolio win!!!'
            else:
                win = 'S&P500 win!!!'
            filename = os.path.join(os.getcwd(), 'compare')
            if not os.path.exists(filename):
                os.makedirs(filename)

            with open('./compare/compare_{}.txt'.format(method), 'a') as f:
                f.write(str(start_time) + "--" + str(end_time) + "  " + 'SP500: ' + str(
                    sp500_mean) + "--" + 'Portfolio: ' + str(portfolio_mean) + "---" + win + '\n')

            print("\033[0;36;m 完成\033[0m \033[0;34;m{}--{}\033[0m  \033[0;36;m时间段比较，开始做图 \033[0m".format(start_time,
                                                                                                         end_time))
            self.plot_performance_compare(day_yield_compare_matrix, start_time, end_time, method)
            if index == 0:
                total_compare_matrix = day_yield_compare_matrix
                total_compare_matrix_convert_one = period_day_matrix_convert_one
            else:
                total_compare_matrix = pd.concat([total_compare_matrix, day_yield_compare_matrix])
                total_compare_matrix_convert_one = pd.concat(
                    [total_compare_matrix_convert_one, period_day_matrix_convert_one])

        return total_compare_matrix, total_compare_matrix_convert_one

    @staticmethod
    def plot_performance_compare(yield_matrix, start_time, end_time, method, yield_rate=True):
        sp500 = yield_matrix['SP500'].to_numpy()
        portfolio = yield_matrix['Portfolio'].to_numpy()
        period = yield_matrix["Period"].to_numpy()
        # plt.figure(figsize=(15, 9))
        plt.style.use('seaborn-dark')

        fig, ax = plt.subplots()

        plt.rcParams['savefig.dpi'] = 300 
        plt.rcParams['figure.dpi'] = 300  
        plt.rcParams['axes.unicode_minus'] = False  
        ax.plot(period, sp500, label='S&P500')
        ax.plot(period, portfolio, label='portfolio')

        if yield_rate:
            ax.set(xlabel='Date', ylabel='Yield by Day',
                   title="S&P500 and Portfolio by {}_{}-{}".format(method, start_time, end_time))
        else:
            ax.set(xlabel='Date', ylabel='Asset Normalization',
                   title="S&P500 and Portfolio by {}_{}-{}".format(method, start_time, end_time))
        ax.grid()
        ax.legend()

        filename = os.path.join(os.getcwd(), 'images1')
        if not os.path.exists(filename):
            os.makedirs(filename)
        if yield_rate:
            plt.savefig("./images1/Investment Return Comparison of S&P500 and Portfolio by {}_{}-{}".format(method, start_time, end_time), dpi=300)
        else:
            plt.savefig("./images1/Investment Return Comparison of S&P500 and Portfolio by {}_{}-{}".format(method, start_time, end_time), dpi=300)
        plt.close()
        # plt.show()


if __name__ == '__main__':
    matplotlib.use('Agg')

    invent_strate = InvestmentStrategy()

    # get_six_months_portfolio_weight
    invent_strate.save_weights_markowitz()
    invent_strate.save_weights_montocarlo()

    # method = "Markowitz"
    # method = "MontoCarlo"
    # method = "MontoCarlo_alpha0"
    for method in ["Markowitz", "MontoCarlo", "MontoCarlo_alpha0"]:
        total_compare_yield_matrix, total_compare_matrix_convert_one = invent_strate.compare_performance(method=method)
        sp500_mean_total = np.mean(total_compare_yield_matrix['SP500'].to_numpy())
        portfolio_mean_total = np.mean(total_compare_yield_matrix['Portfolio'].to_numpy())
        if sp500_mean_total < portfolio_mean_total:
            win = 'Portfolio win!!!'
        else:
            win = 'S&P500 win!!!'

        with open('./compare/compare_{}.txt'.format(method), 'a') as f:
            f.write("Total Ave.：" + str(20150105) + "--" + str(20191230) + "  " + 'SP500: ' + str(
                sp500_mean_total) + "--" + 'Portfolio: ' + str(portfolio_mean_total) + "---" + win + '\n')

        
        invent_strate.plot_performance_compare(total_compare_yield_matrix, 20150105, 20191230, method)
        invent_strate.plot_performance_compare(total_compare_matrix_convert_one, 20150105, 20191230, method,
                                               yield_rate=False)


Computing20170703--20220630weights: 100%|██████| 10/10 [00:00<00:00, 100.55it/s]


 Initial Calculation of Portfolio Weights Using the Strategy：  Markowitz Portfolio 
 Complete Markowitz Portfolio Optimal Weight Quadratic Programming Solution with the Minimum Variance Value：  1.6140103280698902e-05 
 Initial Calculation of Portfolio Weights Using the Strategy：  Markowitz Portfolio 
 Complete Markowitz Portfolio Optimal Weight Quadratic Programming Solution with the Minimum Variance Value：  1.7554799857823897e-05 
 Initial Calculation of Portfolio Weights Using the Strategy：  Markowitz Portfolio 
 Complete Markowitz Portfolio Optimal Weight Quadratic Programming Solution with the Minimum Variance Value：  1.8121360561215513e-05 
 Initial Calculation of Portfolio Weights Using the Strategy：  Markowitz Portfolio 
 Complete Markowitz Portfolio Optimal Weight Quadratic Programming Solution with the Minimum Variance Value：  1.7516747025799126e-05 
 Initial Calculation of Portfolio Weights Using the Strategy：  Markowitz Portfolio 
 Complete Markowitz Portfolio Optimal Weight

Computing20130102--20171229weights:   0%|                | 0/10 [00:00<?, ?it/s]

 Start Calculation of Portfolio Weights Using the Strategy：  Monto Carlo Solve for the Maximum Sharpe Ratio 



100%|██████████████████████████████████| 50000/50000 [00:01<00:00, 37347.85it/s]
/var/folders/xb/n8vz2nsx6xv6l4kn_m4mzkf40000gn/T/ipykernel_16828/167817763.py:147: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(8, 6))


...


100%|██████████████████████████████████| 50000/50000 [00:01<00:00, 38362.92it/s]


 finished Monto Carlo Strategy Weight Computation 


Computing20170703--20220630weights: 100%|███████| 10/10 [00:43<00:00,  4.31s/it]

 Complete Capital Market Line Plotting. 
 saved 
 Start Comparison with S&P 500 Performance, Using the Comparison Strategy is   Markowitz 


 完成 20180102--20180629  时间段比较，开始做图 
 完成 20180702--20181231  时间段比较，开始做图 
 完成 20190102--20190628  时间段比较，开始做图 
 完成 20190701--20191231  时间段比较，开始做图 
 完成 20200102--20200630  时间段比较，开始做图 
 完成 20200701--20201231  时间段比较，开始做图 
 完成 20210104--20210630  时间段比较，开始做图 
 完成 20210701--20211231  时间段比较，开始做图 
 完成 20220103--20220630  时间段比较，开始做图 
 完成 20220701--20221229  时间段比较，开始做图 
 Start Comparison with S&P 500 Performance, Using the Comparison Strategy is   MontoCarlo 
 完成 20180102--20180629  时间段比较，开始做图 
 完成 20180702--20181231  时间段比较，开始做图 
 完成 20190102--20190628  时间段比较，开始做图 
 完成 20190701--20191231  时间段比较，开始做图 
 完成 20200102--20200630  时间段比较，开始做图 
 完成 20200701--20201231  时间段比较，开始做图 
 完成 20210104--20210630  时间段比较，开始做图 
 完成 20210701--20211231  时间段比较，开始做图 
 完成 20220103--20220630  时间段比较，开始做图 
 完成 20220701--20221229  时间段比较，开始做图 
 Start Comparison with S&P 500 Performance, Using the Comparison Strategy is   MontoCarlo_alpha0 
 完成 20180102--20180629  时间段比较，开始做图 
 完成 20180702--20181231  时间段比较，开始做图 
 完成 20190102--20190